In [ ]:
import   os
import   sys
sys.path.insert(0, os.path.abspath('.'))
from     ismrmrdUtils   import   rawMRutils
import   numpy          as       np

## First - read in (fully sampled) calibration data from array coil

In [ ]:
# Put data reads from disk in their own cells as these don't need to be repeated
calDataArrayHeader, calDataArray = rawMRutils.returnHeaderAndData('./ScanArchive_20190529_090502374_converted.h5')

In [ ]:
calDataArrayImageSpace = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(calDataArray, axes=(1,2)), axes=(1,2)), axes=(1,2))
# rawMRutils.computeAndPlot(calDataArrayImageSpace, quant='phase', coil=6)

## Now - read in data from body coil.

In [ ]:
calDataBodyHeader, calDataBody = rawMRutils.returnHeaderAndData('./ScanArchive_20190529_090622648_converted.h5')

In [ ]:
# imageSpace = np.fft.fftshift(np.fft.fft2(calDataBody, axes=(1,2)), axes=(1,2))
calDataBodyImageSpace = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(calDataBody, axes=(1,2)), axes=(1,2)), axes=(1,2))
# rawMRutils.computeAndPlot(calDataBodyImageSpace, quant='phase')

## Read in noise data from array coil.

In [ ]:
noiseDataArrayHeader, noiseDataArray = rawMRutils.returnHeaderAndData('./ScanArchive_20190529_090902358_converted.h5')

### Create an artificially sub-sampled (accelerated) set of data from the array coil data (R = 2)

In [ ]:
subSampledKSpace = np.zeros((calDataArray.shape), dtype=np.complex64)
subSampledKSpace[:, :, ::2, :, :] = calDataArray[:, :, ::2, :, :]
aliasedImageSpace = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(subSampledKSpace, axes=(1,2)), axes=(1,2)), axes=(1,2))
# rawMRutils.computeAndPlot(aliasedImageSpace, quant='mag', coil=-1)

In [ ]:
scaledCalDataArray = np.divide (calDataArrayImageSpace, abs(calDataBodyImageSpace))

In [ ]:
rawMRutils.computeAndPlot(scaledCalDataArray, quant='mag', coil=0)

In [ ]:
def regularizeSensitivityMap (scaledArrayData):

   '''
      This routine should take fully sampled array coil data, already
      scaled with either its own square root of sum of squares of the
      individual coil images, or body coil data.

      The approach below computes a cosine window over the supported
      k-space region, converts the image data back into k-space data,
      applies the cosine weighting, converts back into image space,
      and is then returned.
   '''

   # First, compute cosine kernel over k-space
   smoothingKernelData = np.zeros(scaledArrayData.shape, dtype=np.float64)
   smoothingKernelData.shape

   resX = smoothingKernelData.shape[1]
   resY = smoothingKernelData.shape[2]

   midX = resX/2
   midY = resY/2

   if (resX > resY):
      maxRadius = resY / 2.0
   else:
      maxRadius = resX / 2.0

   for y in range(0, resY):
      for x in range (0, resX):
         offX = np.double(x) - np.double (midX)
         offY = np.double(y) - np.double (midY)

         radius = np.sqrt ((offX * offX) + (offY * offY))

         if (radius < maxRadius):
            smoothingKernelData[:, x, y, :, :] = np.cos(np.pi * 0.5 * radius/maxRadius)
         else:
            smoothingKernelData[:, x, y, :, :] = 0.0

   # computeAndPlot(smoothingKernelData)

   # Convert image space sensitivity map to k-space, and apply the above
   # computed (here - cosine) window
   windowedKSpace = (np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(scaledArrayData, axes=(1,2)), axes=(1,2)), axes=(1,2)) * smoothingKernelData)

   # fft back to image space and return smoothed map
   return np.fft.fftshift(np.fft.fft2(np.fft.fftshift(windowedKSpace, axes=(1,2)), axes=(1,2)), axes=(1,2))


In [ ]:
windowedCalDataArrayImageSpace = regularizeSensitivityMap(scaledCalDataArray)
rawMRutils.computeAndPlot(windowedCalDataArrayImageSpace, coil=31)